# Running the pre-night briefing dashboard within a notebook

## Notebook perparation

### Load jupyter extensions

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
import warnings
import math
import panel as pn
import numpy as np

In [ ]:
from astropy.time import Time, TimeDelta

In [ ]:
from rubin_sim.scheduler.example import example_scheduler
from rubin_sim.scheduler import sim_runner
from rubin_sim.scheduler.model_observatory import ModelObservatory

In [ ]:
%aimport schedview
%aimport schedview.app.prenight

In [ ]:
# from schedview.app.prenight import prenight_app

### Further preparation of the notebook

In [ ]:
pn.extension()

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [ ]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)

## Configuration and initial configuration

Setting `keep_rewards` to `True` results in a dashboard that includes plots of rewards.

In [ ]:
keep_rewards = True

Set the start date, scheduler, and observatory for the night:

In [ ]:
observatory = ModelObservatory()

Set `evening_mjd` to the integer calendar MJD of the local calendar day on which sunset falls on the night of interest.

In [ ]:
evening_mjd = Time("2025-01-01").mjd

If we just use this day as the start and make the simulation duration 1 day, the begin and end of the simulation will probably begin in the middle on one night and end in the middle of the next.
Instead, find the sunset and sunrise of the night we want using the almanac, and use these to determine our start time and duration.

In [ ]:
# If the date represents the local calendar date at sunset, we need to shift by the longitude in units of days
this_night = (
    np.floor(observatory.almanac.sunsets["sunset"] + observatory.site.longitude / 360)
    == evening_mjd
)

mjd_start = observatory.almanac.sunsets[this_night]["sun_n12_setting"][0]
mjd_end = observatory.almanac.sunsets[this_night]["sunrise"][0]

night_duration = mjd_end - mjd_start
Time(mjd_start, format="mjd").iso, night_duration

In [ ]:
observatory = ModelObservatory(mjd_start=mjd_start)

In [ ]:
scheduler = example_scheduler(mjd_start=mjd_start)

## Run a simulation and create the app instance

For this example, simulate starting the default first day of observing:

In [ ]:
if not keep_rewards:
    observatory, scheduler, observations = sim_runner(
        observatory, scheduler, mjd_start=mjd_start, survey_length=night_duration
    )
    app = schedview.app.prenight.prenight_app(observatory, scheduler, observations)
else:
    scheduler.keep_rewards = True
    observatory, scheduler, observations, reward_df, obs_rewards = sim_runner(
        observatory,
        scheduler,
        mjd_start=mjd_start,
        survey_length=night_duration,
        record_rewards=True,
    )
    app = schedview.app.prenight.prenight_app(
        observatory,
        scheduler,
        observations,
        reward_df=reward_df,
        obs_rewards=obs_rewards,
    )

## Display the dashboard

Let's look at the last (and only) full night we simulated:

In [ ]:
app

# Adjusting plot parameters beyond what the explore interface does

Build an independent explorer.

In future, the pre-night briefing app code will provide direct access to the instance of the explorer displayed (see PREOPS-3412).

For the time being, you can reproduce the explorer independently, and customize it from there:

Start by getting the data set used by the explorer:

In [ ]:
import pandas as pd
import hvplot
import hvplot.pandas
from rubin_sim.scheduler.utils import SchemaConverter

schema_converter = SchemaConverter()
visits = schema_converter.obs2opsim(observations)
visits["start_date"] = pd.to_datetime(
    visits["observationStartMJD"] + 2400000.5, origin="julian", unit="D", utc=True
)
visits.describe().T

In [ ]:
ex = hvplot.explorer(visits, kind="scatter", x="start_date", y="airmass", by=["note"])
ex

Use the explorer GUI above to get the plot as close as you can to what you want, then use the cell below to capture the python needed to generate that plot, and make further adjustments as necessary.

In [ ]:
ex.plot_code()

In [ ]:
visits.hvplot(
    by=["note"],
    kind="scatter",
    x="start_date",
    ylim=(3, 1),
    y=["airmass"],
)